In [45]:
import pandas as pd
import numpy as np 


In [46]:
df = pd.read_csv('../data/raw/no_stock.csv')

/var/folders/fm/g7cx2fj16_7c2vf36zcb24_40000gp/T/ipykernel_29722/4261824222.py:1: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/raw/no_stock.csv')


In [47]:
df.head()

,country_code,order_no,prime_line_no,status_description,line_status_code,order_date,order_type,service_type,sales_channel,item_id,...,time_to_min_notification_date_days,time_to_promised_appt_start_date_mins,time_to_promised_appt_start_date_hours,time_to_promised_appt_start_date_days,time_to_dispatch_date_mins,time_to_dispatch_date_hours,time_to_dispatch_date_days,deviation,deviation_rate_1month,stock_related_deviation_rate_1month
0,FR,1554124694,8,Allocated,1500.02,2025-10-17 19:08:53+00:00,HOME_DELIVERY,HOME_DELIVERY,INTERNET,ART40586508,...,3.493657,6590.866667,109.847778,4.576991,5870.866667,97.847778,4.076991,0,8.521334,3.121213
1,DE,1555179093,4,Sent for Fulfillment,3200.02,2025-10-22 10:39:52+00:00,CLICK_COLLECT,PUP,INTERNET,ART20447269,...,0.925093,6972.133333,116.202222,4.841759,3492.133333,58.202222,2.425093,0,7.157930,2.268342
2,ES,1555181659,22,Created,1100.00,2025-10-22 10:55:14+00:00,HOME_DELIVERY,HOME_DELIVERY,INTERNET,ART00573188,...,1.003021,2584.350000,43.072500,1.794687,1744.350000,29.072500,1.211354,0,4.344047,0.558623
3,DE,1555158732,1,Created,1100.00,2025-10-22 09:02:28+00:00,HOME_DELIVERY,HOME_DELIVERY,INTERNET,ART10391833,...,1.706424,4077.250000,67.954167,2.831424,2937.250000,48.954167,2.039757,0,4.975760,1.069925
4,FR,1553176353,65,Allocated,1500.02,2025-10-13 15:42:55+00:00,HOME_DELIVERY,HOME_DELIVERY,REMOTE_KITCHEN,ART40368770,...,30.832812,51359.250000,855.987500,35.666146,47099.250000,784.987500,32.707813,0,16.925044,0.900114


In [48]:
#print every column name with number and type
for i, col in enumerate(df.columns):
    print(i, col, df[col].dtype)

0 country_code object
1 order_no int64
2 prime_line_no int64
3 status_description object
4 line_status_code float64
5 order_date object
6 order_type object
7 service_type object
8 sales_channel object
9 item_id object
10 ship_node object
11 ordered_qty float64
12 line_total_euros float64
13 transport_method_type object
14 item_group_code object
15 latest_dispatch_date object
16 work_order_no float64
17 ingestion_time object
18 modifyts object
19 cancellation_date object
20 delivery_date object
21 line_status_date object
22 payment_type object
23 payment_due_at object
24 payment_fully_paid_date object
25 customer_type object
26 number_of_DS int64
27 number_of_PS int64
28 promised_update_DS int64
29 promised_update_PS int64
30 parent_prime_line_no_DS float64
31 level_of_service_DS object
32 promised_appt_start_date_DS object
33 promised_appt_end_date_DS object
34 WO_creation_date object
35 WO_acknowledgement_date object
36 min_notification_time float64
37 next_line_status_date object
38 

In [49]:
#print value counts also for nans
print(df['event_type'].value_counts(dropna=False))

event_type
NaN              5062142
Auto-recovery       6599
Deviation           2037
Name: count, dtype: int64


In [50]:
#print nans for every column with percentage in descending order
nan_counts = []
for col in df.columns:
    nans = df[col].isna().sum()
    if nans > 0:
        nan_counts.append((col, nans, nans/len(df)*100))

nan_counts.sort(key=lambda x: x[1], reverse=True)

for col, nans, perc in nan_counts:
    print(f"{col}: {nans} nans ({perc:.2f}%)")

reason_caused: 5068741 nans (99.96%)
ARF_type: 5064179 nans (99.87%)
event_type: 5062142 nans (99.83%)
event_time: 5062142 nans (99.83%)
deviation_reason_code: 5062142 nans (99.83%)
deviation_occurence: 5062142 nans (99.83%)
cancellation_date: 4775255 nans (94.17%)
days_since_last_stockout: 4050223 nans (79.87%)
average_daily_stock_change_rate: 1239398 nans (24.44%)
payment_type: 873464 nans (17.23%)
payment_due_at: 873464 nans (17.23%)
item_id_service_level: 818625 nans (16.14%)
payment_fully_paid_date: 786439 nans (15.51%)
days_since_last_replenishment: 728755 nans (14.37%)
estimated_days_till_next_stockout: 657492 nans (12.97%)
current_daily_stock_change_rate: 646025 nans (12.74%)
average_daily_stock_change_rate_replenishment: 646023 nans (12.74%)
stock_variance_4_weeks: 644361 nans (12.71%)
availability_qty: 642350 nans (12.67%)
stockout_count_28_days: 642350 nans (12.67%)
replenishment_count_28_days: 642350 nans (12.67%)
modify_date_time: 642344 nans (12.67%)
average_stock_4_weeks

In [51]:
#drop columns with more than 90% nans
df = df.loc[:, df.isna().mean() < 0.9]

In [59]:
# for "days_since_last_stockout" column, make all nans max value + 1
max_days = df['days_since_last_stockout'].max()
df['days_since_last_stockout'] = df['days_since_last_stockout'].fillna(max_days + 1)

In [60]:
df['item_id_service_level'].value_counts(dropna=False)

item_id_service_level
3.0    1591235
2.0    1226576
4.0     986096
NaN     818625
1.0     448246
Name: count, dtype: int64

In [61]:
#calculate the ratio of nans with deviation=1 for each column that has nans in descending order
nan_deviation_counts = []
for col in df.columns:
    if df[col].isna().sum() > 0:
        nan_deviation_1 = df[df['deviation'] == 1][col].isna().sum()
        total_deviation_1 = len(df[df['deviation'] == 1])
        ratio_1 = nan_deviation_1 / total_deviation_1 if total_deviation_1 > 0 else 0
        # print(f"{col}: Deviation=1 NaN Ratio: {ratio_1:.4f}")
        nan_deviation_counts.append((col, ratio_1))

nan_deviation_counts.sort(key=lambda x: x[1], reverse=True)
for col, ratio_1 in nan_deviation_counts:
    print(f"{col}: Deviation=1 NaN Ratio: {ratio_1:.4f}")

delivery_date: Deviation=1 NaN Ratio: 0.4267
item_id_service_level: Deviation=1 NaN Ratio: 0.1877
average_daily_stock_change_rate: Deviation=1 NaN Ratio: 0.1474
payment_type: Deviation=1 NaN Ratio: 0.1293
payment_due_at: Deviation=1 NaN Ratio: 0.1293
forecast_volume_next_day: Deviation=1 NaN Ratio: 0.0732
forecast_stock_next_day: Deviation=1 NaN Ratio: 0.0713
forecast_volume_current_day: Deviation=1 NaN Ratio: 0.0691
forecast_volume_previous_day: Deviation=1 NaN Ratio: 0.0691
forecast_stock_current_day: Deviation=1 NaN Ratio: 0.0673
forecast_stock_previous_day: Deviation=1 NaN Ratio: 0.0673
payment_fully_paid_date: Deviation=1 NaN Ratio: 0.0610
days_since_last_replenishment: Deviation=1 NaN Ratio: 0.0406
min_notification_date: Deviation=1 NaN Ratio: 0.0350
time_to_min_notification_date_mins: Deviation=1 NaN Ratio: 0.0350
time_to_min_notification_date_hours: Deviation=1 NaN Ratio: 0.0350
time_to_min_notification_date_days: Deviation=1 NaN Ratio: 0.0350
estimated_days_till_next_stockout: